In [1]:
#This notebook uses an API that was built utilizing the ESPN Fantasy Football API 
#The documentation can be found here:
#https://github.com/cwendt94/ff-espn-api

%matplotlib inline
#The necessary packages
import pandas as pd
import numpy as np
import seaborn as sns
from ff_espn_api import League

#Info:
league_id=4570371
season=2019
#team='10'

In [2]:
league=League(league_id,2019)
#Possible info
myTeam=league.teams[9]

In [91]:
#At the moment, the draft hasn't happened yet. 
#Filter the free agents by position and ranking
positions=['QB','RB','WR','TE','FLEX','D/ST','K']
positionDict={}
for pos in positions:
    playerList=league.free_agents(position=pos)
    rankings=[p.posRank for p in playerList]
    pid=[p.playerId for p in playerList]
    pTeam=[p.proTeam for p in playerList]
    playerDat=pd.DataFrame(list(zip(playerList,pid,pTeam,rankings)),
                           columns=['Player','ID','Team','Ranking']).sort_values(by=['Ranking'])
    playerDat=playerDat[playerDat['Ranking']>0]
    #Redo the index numbers 
    playerDat.index=range(len(playerDat))
    positionDict[pos]=playerDat
    

In [44]:
#Historical data: all credit goes to this page:
#https://github.com/derek-adair/nflgame
#The documentation for all of the functionality here:
#http://nflgame.derekadair.com/
#However, the more idiot proof guide is here (bless this person!!)
#https://github.com/BurntSushi/nflgame/wiki/Tutorial-for-non-programmers:-Installation-and-examples
import nflgame

#Games from 2018
games=nflgame.games(2018)
players=nflgame.combine_play_stats(games)
player_list=[p for p in players]
player_names=[p.name for p in players]

In [95]:
#Make a dataframe of the 
#for p in players:
#    print(p,p.playerid,p.stats)

#Can we add more detailed stats to the list of players?

fpr 
df_player={}
for p in positionDict[pos]['Player']:
    playerName=p.name
    print(playerName)
    finit=playerName[0]
    lastname=playerName.split(" ")[1]
    nameFormat='{:}.{:}'.format(finit,lastname)
    #Check if player is in list
    pcheck=(nameFormat in player_names)
    if pcheck==True:
        pIndex=player_names.index(nameFormat)
        playerStats=player_list[pIndex].stats
        df_player[playerName]=playerStats




Patrick Mahomes
Matt Ryan
Ben Roethlisberger
Deshaun Watson
Andrew Luck
Aaron Rodgers
Jared Goff
Drew Brees
Russell Wilson
Dak Prescott
Philip Rivers
Kirk Cousins
Cam Newton
Tom Brady
Mitchell Trubisky
Baker Mayfield
Eli Manning
Derek Carr
Case Keenum
Matthew Stafford
Josh Allen
Jameis Winston
Carson Wentz
Marcus Mariota
Andy Dalton
Blake Bortles
Sam Darnold
Ryan Fitzpatrick
Lamar Jackson
Ryan Tannehill
Alex Smith
Joe Flacco
Nick Mullens
Josh Rosen
Nick Foles
Jimmy Garoppolo
Taysom Hill
Nathan Peterman
Sam Bradford
Teddy Bridgewater
Robert Griffin III
Mason Rudolph
Jacoby Brissett


In [96]:
df_player

{'Patrick Mahomes': OrderedDict([('passing_yds', 5097),
              ('passing_att', 580),
              ('passing_cmp', 383),
              ('passing_cmp_air_yds', 2489),
              ('rushing_yds', 272),
              ('rushing_att', 60),
              ('passing_tds', 50),
              ('passing_incmp', 197),
              ('passing_incmp_air_yds', 2771),
              ('passing_sk_yds', -171),
              ('passing_sk', 26),
              ('fumbles_tot', 9),
              ('fumbles_notforced', 7),
              ('fumbles_rec_yds', -4),
              ('fumbles_rec', 5),
              ('rushing_tds', 2),
              ('penalty_yds', 38),
              ('penalty', 5),
              ('passing_int', 12),
              ('passing_twopta', 1),
              ('passing_twoptmissed', 1),
              ('fumbles_forced', 2),
              ('fumbles_lost', 2),
              ('rushing_twopta', 1),
              ('rushing_twoptm', 1),
              ('name', 'P.Mahomes')]),
 'Matt Ryan': Ord

In [101]:
#Another version:
#Get the JSON table in easily readable format
#Basic team info
url='https://fantasy.espn.com/apis/v3/games/ffl/seasons/2019/segments/0/leagues/{:}'.format(league_id)
r=requests.get(url)
dat=r.json()

In [ ]:
#Get the weekly matchup scores
slotcodes={0:'QB',2:'RB',4:'WR',
          6:'TE',16:'Def',17:'K',
          20:'Bench',21:'IR',23:'Flex'
          }
url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + \
      str(season) + '/segments/0/leagues/' + str(league_id) + \
      '?view=mMatchup&view=mMatchupScore'


In [ ]:
data=[]
print('Week ',end='')
for week in range(1,2):
    print(week,end=' ')
    r=requests.get(url,params={'scoringPeriodId':week})
    d=r.json()
    for tm in d['teams']:
        tmid=tm['id']
        for p in tm['roster']['entries']:
            name=p['playerPoolEntry']['player']['fullName']
            slot=p['lineupSlotId']
            pos=slotcodes[slot]
            
            inj='NA'
            try:
                inj=p['playerPoolEntry']['player']['injuryStatus']
            except:
                pass
        proj,act=None,None
        for stat in p['playerPoolEntry']['player']['stats']:
            if stat['scoringPeriodId']!=week:
                continue
            if stat['statSourceId'] == 0:
                act = stat['appliedTotal']
            elif stat['statSourceId']==1:
                proj=stat['appliedTotal']
        data.append([
                week, tmid, name, slot, pos, inj, proj, act
        ])

In [ ]:
data = pd.DataFrame(data, 
                    columns=['Week', 'Team', 'Player', 'Slot', 
                             'Pos', 'Status', 'Proj', 'Actual'])